# Set Environment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime
from itertools import product

from google.colab import auth
from google.cloud import bigquery
from gspread_dataframe import set_with_dataframe

import gspread
from oauth2client.client import GoogleCredentials
from google.auth import default
creds, _ = default()

import pandas_gbq
from tqdm import tqdm

auth.authenticate_user()
print('Authenticated')

In [ ]:
# input parameters
# pre start date and end date
pre_start_date = '2024-08-31'
pre_end_date = '2024-09-08'

# post start date and end date
post_start_date = '2024-10-05'
post_end_date = '2024-10-13'

# declare each parameter as correct data type
query_params = [
    bigquery.ScalarQueryParameter("pre_start_date", "DATE", pre_start_date),
    bigquery.ScalarQueryParameter("pre_end_date", "DATE", pre_end_date),
    bigquery.ScalarQueryParameter("post_start_date", "DATE", post_start_date),
    bigquery.ScalarQueryParameter("post_end_date", "DATE", post_end_date)
]

job_config = bigquery.QueryJobConfig(query_parameters=query_params)

# Project configuration
project_id = 'database_name'
client = bigquery.Client(project=project_id)

# Insert to Google sheet
# Remember to change G-sheet URL
spreadsheet_url = 'https://docs.google.com/spreadsheets/link'
worksheet_title = 'data'

# Balance Treatment Group and Control Group
is_balance = True


# Get Voucher Experiment Data

In [ ]:
df = client.query(f'''

DECLARE campaign_label STRING DEFAULT "AAA_voucher_AFV_CG&TG";

WITH segmentation_label AS (
  SELECT
    cust_id,
    afv_group
  FROM customer_segmentation__snapshot
  WHERE partition_date_utc = "2024-09-28"
),

tg_cg_user_list AS (
  SELECT
    DISTINCT
    carousel_user_list.user_group,
    dimension,
    cust_id
  FROM carousel_user_list
  LEFT JOIN segmentation_label USING (cust_id)
  WHERE url_key IN ("OctpureTG","202410_NV_SH_C_AAA_CG")
),

pd_orders_table AS (
  SELECT
    cust_id,

    /* pre */
    COUNT(CASE WHEN DATE(expected_delivery_at_local) BETWEEN @pre_start_date AND @pre_end_date THEN uuid END) AS pre_orders,
    SUM(CASE WHEN DATE(expected_delivery_at_local) BETWEEN @pre_start_date AND @pre_end_date THEN agg_accounting.gfv_local END) AS pre_gfv_local,
    SUM(CASE WHEN DATE(expected_delivery_at_local) BETWEEN @pre_start_date AND @pre_end_date THEN agg_accounting.gmv_local END) AS pre_gmv_local,
    SUM(CASE
          WHEN DATE(expected_delivery_at_local) BETWEEN @pre_start_date AND @pre_end_date
          THEN IFNULL(ABS(accounting.vouchers_plus_vat_local), 0) * IFNULL(agg_vouchers.attributions_foodpanda_ratio, 0) / 100
        END) AS pre_own_voucher_local,

    /* post */
    COUNT(CASE WHEN DATE(expected_delivery_at_local) BETWEEN @post_start_date AND @post_end_date THEN uuid END) AS post_orders,
    SUM(CASE WHEN DATE(expected_delivery_at_local) BETWEEN @post_start_date AND @post_end_date THEN agg_accounting.gfv_local END) AS post_gfv_local,
    SUM(CASE WHEN DATE(expected_delivery_at_local) BETWEEN @post_start_date AND @post_end_date THEN agg_accounting.gmv_local END) AS post_gmv_local,
    SUM(CASE
          WHEN DATE(expected_delivery_at_local) BETWEEN @post_start_date AND @post_end_date
          THEN IFNULL(ABS(accounting.vouchers_plus_vat_local), 0) * IFNULL(agg_vouchers.attributions_foodpanda_ratio, 0) / 100
        END) AS post_own_voucher_local,

  FROM orders
  LEFT JOIN vdr_svc_vendors
         ON pd_vendor_uuid = vdr_svc_vendors.uuid
  LEFT JOIN agg_accounting
         ON uuid = agg_accounting.uuid
  LEFT JOIN accounting
         ON accounting.is_order_last_entry
  LEFT JOIN agg_vouchers
         ON code = agg_vouchers.order_code
  WHERE created_date_utc >= DATE_SUB(@pre_start_date, INTERVAL 3 DAY)
  GROUP BY 1
)

SELECT
  campaign_label,
  user_group,
  cust_id,
  dimension,

  /* pre metrics */
  CASE WHEN pre_orders > 0 THEN 1 ELSE 0 END AS pre_active,
  COALESCE(pre_orders, 0) AS pre_orders,
  COALESCE(pre_gfv_local, 0) AS pre_gfv_local,
  SAFE_DIVIDE(pre_gfv_local, pre_orders) AS pre_afv,
  COALESCE(pre_gmv_local, 0) AS pre_gmv_local,
  COALESCE(pre_own_voucher_local, 0) AS pre_own_voucher_local,

  /* post metrics */
  CASE WHEN post_orders > 0 THEN 1 ELSE 0 END AS post_active,
  COALESCE(post_orders, 0) AS post_orders,
  COALESCE(post_gfv_local, 0) AS post_gfv_local,
  SAFE_DIVIDE(post_gfv_local, post_orders) AS post_afv,
  COALESCE(post_gmv_local, 0) AS post_gmv_local,
  COALESCE(post_own_voucher_local, 0) AS post_own_voucher_local,

FROM tg_cg_user_list
LEFT JOIN pd_orders_table
       ON cust_id = cust_id

''', job_config=job_config).to_dataframe()
print("SQL completed! \n")

SQL completed! 



# Check data

In [ ]:
df.isna().sum()

In [ ]:
df.dtypes

In [ ]:
df.describe()

# Set Up formula for T testing, G-sheet storage



In [ ]:
def run_ttest_two_tailed(control_group, treatment_group):
  ''' Run A/B testing '''
  control_mean = control_group.mean()
  treatment_mean = treatment_group.mean()
  control_sum = control_group.sum()
  treatment_sum = treatment_group.sum()

  t_statistic, p_value = stats.ttest_ind(control_group, treatment_group)

  mean_diff = np.mean(control_group) - np.mean(treatment_group)
  std_error = np.sqrt(np.var(control_group)/len(control_group) + np.var(treatment_group)/len(treatment_group))

  # confidence level
  confidence_level = 0.95
  alpha = 1 - confidence_level

  # df
  degrees_of_freedom = len(control_group) + len(treatment_group) - 2

  # t critical
  t_critical = stats.t.ppf(1 - alpha / 2, degrees_of_freedom)

  # SE
  standard_error = np.sqrt(np.var(control_group) / len(control_group) + np.var(treatment_group) / len(treatment_group))

  # lower bound, upper bound
  mean_difference = np.mean(control_group) - np.mean(treatment_group)
  margin_of_error = t_critical * standard_error
  lower_bound = mean_difference - margin_of_error
  upper_bound = mean_difference + margin_of_error
  confidence_interval = (lower_bound, upper_bound)
  significant = p_value <= 0.05

  return control_mean, treatment_mean, control_sum, treatment_sum, t_statistic, p_value, significant, confidence_interval



def get_dynamic_df(columns):
  ''' The dataframe dimension might be changed for each reviews, set up the dynamic function to get the result df columns '''
  dynamic_columns = columns
  front_columns = ['campaign_name']
  back_columns = ['metric', 'method', 'control_size', 'treatment_size', 'control_active', 'treatment_active', 'control_mean', 'treatment_mean', 'TG-CG_mean', 'control_sum', 'treatment_sum', 't-statistic', 'p-value', 'significant', 'confidence_interval']
  all_columns = front_columns + dynamic_columns + back_columns
  df = pd.DataFrame(columns=all_columns)

  return df



def processing_df(df, dynamic_dimension):
  ''' Add information to the dataframe including ['period', 'data_start_date', 'data_end_date'] '''
  df['period'] = np.where(df['metric'].str.contains('pre_'), 'Pre', np.where(df['metric'].str.contains('post_'), 'Post', ''))
  df['data_start_date'] = np.where(df['period'].str.contains('Pre'), pre_start_date, np.where(df['period'].str.contains('Post'), post_start_date, ''))
  df['data_end_date'] = np.where(df['period'].str.contains('Pre'), pre_end_date, np.where(df['period'].str.contains('Post'), post_end_date, ''))
  df['metric'] = df['metric'].str.replace('pre_', '').str.replace('post_', '')


  df = df.loc[:,['campaign_name'] + dynamic_dimension + ['period', 'metric',
                              'method', 'control_size', 'treatment_size', 'control_active', 'treatment_active',
                              'control_mean', 'treatment_mean', 'TG-CG_mean', 'control_sum', 'treatment_sum', 't-statistic', 'p-value', 'significant',
                              'confidence_interval', 'data_start_date', 'data_end_date']]

  return df



def df_insert_to_google_sheet(df, spreadsheet_url, worksheet_title):
  ''' Insert result data to the google sheet '''
  gc = gspread.authorize(creds)
  workbook = gc.open_by_url(spreadsheet_url)
  try:
    worksheet = workbook.worksheet(title=worksheet_title)
    workbook.del_worksheet(worksheet=worksheet)
  except:
    pass

  worksheet = workbook.add_worksheet(title=worksheet_title, rows=1, cols=1)
  set_with_dataframe(worksheet, result_df)

# Check the Dimension & Metrics columns

In [ ]:
fix_dimension = df.columns.values[:3]
dynamic_dimension = df.columns.values[3:4]
dynamic_metrics = df.columns.values[4:]

print(f'All Columns: {df.columns.values} \n')
print(f'fix_dimension: {fix_dimension} \n')
print(f'dynamic_dimension: {dynamic_dimension} \n')
print(f'dynamic_metrics: {dynamic_metrics} \n')

# Conduct Welch's T Test with confidence interval

In [ ]:
df[dynamic_metrics] = df[dynamic_metrics].astype(float)
campaign_name = df['campaign_label'].unique()[0]
result_df = get_dynamic_df(dynamic_dimension.tolist())

combinations = list(product(*(df[col].unique() for col in dynamic_dimension)))

for combo in combinations:
    conditions = [df[col] == val for col, val in zip(dynamic_dimension, combo)]

    condition = pd.Series(True, index=df.index)

    for cond in conditions:
        condition = condition & cond

    seg_data = df[condition]
    control_count = seg_data[seg_data['user_group'] == 'Control Group'].shape[0]
    treatment_count = seg_data[seg_data['user_group'] == 'Treatment Group'].shape[0]

    if is_balance and treatment_count > control_count:
      sampled_treatment = seg_data[seg_data['user_group'] == 'Treatment Group'].sample(n=control_count, random_state=42)
      seg_data = pd.concat([seg_data[seg_data['user_group'] == 'Control Group'], sampled_treatment])

    for metrics_value in dynamic_metrics:
      control_group = seg_data[seg_data['user_group'] == 'Control Group'][metrics_value].dropna().values
      treatment_group = seg_data[seg_data['user_group'] == 'Treatment Group'][metrics_value].dropna().values

      control_mean, treatment_mean, control_sum, treatment_sum, t_statistic, p_value, significant, confidence_interval = run_ttest_two_tailed(control_group, treatment_group)
      control_active = np.count_nonzero(control_group)
      treatment_active = np.count_nonzero(treatment_group)

      append_list = ([campaign_name]
                      + [f'{item}' for item in combo]
                      + [metrics_value, "Two-Tailed", len(control_group), len(treatment_group), control_active, treatment_active,
                          control_mean, treatment_mean, treatment_mean - control_mean, control_sum, treatment_sum,
                          t_statistic, p_value, significant, confidence_interval]
                    )

      result_df.loc[len(result_df)] = append_list

result_df = processing_df(result_df, dynamic_dimension.tolist())

In [ ]:
result_df.isna().sum()

In [ ]:
result_df.describe()

In [ ]:
result_df.head(100)

,campaign_name,dimension,period,metric,method,control_size,treatment_size,control_active,treatment_active,control_mean,treatment_mean,TG-CG_mean,control_sum,treatment_sum,t-statistic,p-value,significant,confidence_interval,data_start_date,data_end_date
0,AAA_voucher_AFV_CG&TG,None,Pre,active,Two-Tailed,0,0,0,0,NaN,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,False,"(nan, nan)",2024-08-31,2024-09-08
1,AAA_voucher_AFV_CG&TG,None,Pre,orders,Two-Tailed,0,0,0,0,NaN,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,False,"(nan, nan)",2024-08-31,2024-09-08
2,AAA_voucher_AFV_CG&TG,None,Pre,gfv_local,Two-Tailed,0,0,0,0,NaN,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,False,"(nan, nan)",2024-08-31,2024-09-08
3,AAA_voucher_AFV_CG&TG,None,Pre,afv,Two-Tailed,0,0,0,0,NaN,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,False,"(nan, nan)",2024-08-31,2024-09-08
4,AAA_voucher_AFV_CG&TG,None,Pre,gmv_local,Two-Tailed,0,0,0,0,NaN,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,False,"(nan, nan)",2024-08-31,2024-09-08
5,AAA_voucher_AFV_CG&TG,None,Pre,own_voucher_local,Two-Tailed,0,0,0,0,NaN,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,False,"(nan, nan)",2024-08-31,2024-09-08
6,AAA_voucher_AFV_CG&TG,None,Post,active,Two-Tailed,0,0,0,0,NaN,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,False,"(nan, nan)",2024-10-05,2024-10-13
7,AAA_voucher_AFV_CG&TG,None,Post,orders,Two-Tailed,0,0,0,0,NaN,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,False,"(nan, nan)",2024-10-05,2024-10-13
8,AAA_voucher_AFV_CG&TG,None,Post,gfv_local,Two-Tailed,0,0,0,0,NaN,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,False,"(nan, nan)",2024-10-05,2024-10-13
9,AAA_voucher_AFV_CG&TG,None,Post,afv,Two-Tailed,0,0,0,0,NaN,NaN,NaN,0.000000e+00,0.000000e+00,NaN,NaN,False,"(nan, nan)",2024-10-05,2024-10-13


# 匯入G-sheet

In [ ]:
df_insert_to_google_sheet(result_df, spreadsheet_url, worksheet_title)